In [1]:
import sys
sys.path.append("../../")

import json
import pandas as pd
from ingestion.generate_ingest import IngestionGenerator
from pyingest.pyingest import PyIngestForStreamlit

In [2]:
username = "neo4j"
password = "password"
database = "neo4j"
uri = "bolt://localhost:7687"

In [3]:
# data_model = """
# {"nodes": [{"label": "Battle", "properties": ["name", "year", "battle_number", "attacker_king", "defender_king", "attacker_1", "attacker_2", "attacker_3", "attacker_4", "defender_1", "defender_2", "defender_3", "defender_4", "attacker_outcome", "battle_type", "major_death", "major_capture", "attacker_size", "defender_size", "attacker_commander", "defender_commander", "summer", "location", "region", "note"], "unique_constraints": ["name"]}, {"label": "King", "properties": ["name"], "unique_constraints": ["name"]}, {"label": "Commander", "properties": ["name"], "unique_constraints": ["name"]}, {"label": "Location", "properties": ["name"], "unique_constraints": ["name"]}, {"label": "Region", "properties": ["name"], "unique_constraints": ["name"]}, {"label": "BattleOutcome", "properties": [], "unique_constraints": ["name"]}, {"label": "BattleType", "properties": [], "unique_constraints": ["name"]}, {"label": "MajorEvent", "properties": [], "unique_constraints": ["name"]}], "relationships": [{"type": "ATTACKED", "properties": [], "unique_constraints": [], "source": "Battle", "target": "King"}, {"type": "DEFENDED", "properties": [], "unique_constraints": [], "source": "Battle", "target": "King"}, {"type": "ATTACKED_WITH", "properties": [], "unique_constraints": [], "source": "Battle", "target": "Commander"}, {"type": "DEFENDED_WITH", "properties": [], "unique_constraints": [], "source": "Battle", "target": "Commander"}, {"type": "TOOK_PLACE_IN", "properties": [], "unique_constraints": [], "source": "Battle", "target": "Location"}, {"type": "LOCATED_IN", "properties": [], "unique_constraints": [], "source": "Location", "target": "Region"}, {"type": "HAD_OUTCOME", "properties": [], "unique_constraints": [], "source": "Battle", "target": "BattleOutcome"}, {"type": "OF_TYPE", "properties": [], "unique_constraints": [], "source": "Battle", "target": "BattleType"}, {"type": "EXPERIENCED_EVENT", "properties": [], "unique_constraints": [], "source": "Battle", "target": "MajorEvent"}]}
# """

In [4]:
# countrys V4
data_model = """
{"nodes": [{"label": "Country", "properties": ["name"], "unique_constraints": ["name"]}, {"label": "Capital", "properties": ["capital"], "unique_constraints": ["capital"]}, {"label": "Currency", "properties": ["currency_name"], "unique_constraints": ["currency_name"]}, {"label": "Region", "properties": ["region"], "unique_constraints": ["region"]}, {"label": "Subregion", "properties": ["subregion"], "unique_constraints": ["subregion"]}, {"label": "Timezone", "properties": ["timezones"], "unique_constraints": ["timezones"]}], "relationships": [{"type": "HAS_CAPITAL", "properties": [], "unique_constraints": [], "source": "Country", "target": "Capital"}, {"type": "USES_CURRENCY", "properties": [], "unique_constraints": [], "source": "Country", "target": "Currency"}, {"type": "BELONGS_TO_REGION", "properties": [], "unique_constraints": [], "source": "Subregion", "target": "Region"}, {"type": "BELONGS_TO_SUBREGION", "properties": [], "unique_constraints": [], "source": "Country", "target": "Subregion"}, {"type": "HAS_TIMEZONE", "properties": [], "unique_constraints": [], "source": "Country", "target": "Timezone"}]}
"""
csv_name = "countries.csv"
csv_dir = "../../data/csv/"

In [5]:
# csv_name = "got-battles.csv"
# csv_dir = "../../data/csv/"

In [6]:
input_dataframe = pd.read_csv(csv_dir+csv_name)

In [7]:
# if a model other than most recent is desired, then uncomment these lines and select the version number appropriately
# model_version_to_use = []
# model_to_use = summarizer.model_history[model_version_to_use-1].dict

gen = IngestionGenerator(data_model=json.loads(data_model),
                         username=username,
                         password=password,
                         database=database,
                         uri=uri,
                         csv_name=csv_name,
                         csv_dir=csv_dir)

model maps:  [{'country_capital': {'source': {'node': 'MATCH (n:Country{name: row.name})'}, 'target': {'node': 'MATCH (n:Capital{capital: row.capital})'}, 'csv': 'countries.csv', 'relationship': {'rel': 'HAS_CAPITAL'}}}]
model maps:  [{'country_capital': {'source': {'node': 'MATCH (n:Country{name: row.name})'}, 'target': {'node': 'MATCH (n:Capital{capital: row.capital})'}, 'csv': 'countries.csv', 'relationship': {'rel': 'HAS_CAPITAL'}}, 'country_currency': {'source': {'node': 'MATCH (n:Country{name: row.name})'}, 'target': {'node': 'MATCH (n:Currency{currency_name: row.currency_name})'}, 'csv': 'countries.csv', 'relationship': {'rel': 'USES_CURRENCY'}}}, {'country_capital': {'source': {'node': 'MATCH (n:Country{name: row.name})'}, 'target': {'node': 'MATCH (n:Capital{capital: row.capital})'}, 'csv': 'countries.csv', 'relationship': {'rel': 'HAS_CAPITAL'}}, 'country_currency': {'source': {'node': 'MATCH (n:Country{name: row.name})'}, 'target': {'node': 'MATCH (n:Currency{currency_name: 

In [8]:
# gen.config_files_list

In [9]:
yaml_string = gen.generate_pyingest_yaml_string()

In [10]:
print(yaml_string)

server_uri: bolt://localhost:7687
admin_user: neo4j
admin_pass: password
database: neo4j
basepath: file:./

pre_ingest:
  - CREATE CONSTRAINT country_name IF NOT EXISTS FOR (n:Country) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT capital_capital IF NOT EXISTS FOR (n:Capital) REQUIRE n.capital IS UNIQUE;
  - CREATE CONSTRAINT currency_currency_name IF NOT EXISTS FOR (n:Currency) REQUIRE n.currency_name IS UNIQUE;
  - CREATE CONSTRAINT region_region IF NOT EXISTS FOR (n:Region) REQUIRE n.region IS UNIQUE;
  - CREATE CONSTRAINT subregion_subregion IF NOT EXISTS FOR (n:Subregion) REQUIRE n.subregion IS UNIQUE;
  - CREATE CONSTRAINT timezone_timezones IF NOT EXISTS FOR (n:Timezone) REQUIRE n.timezones IS UNIQUE;
files:
- chunk_size: 100
  cql: |
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Country {name: row.name})
  url: $BASE/../../data/csv/countries.csv
- chunk_size: 100
  cql: |
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Capital {capital: row.c

In [10]:
outputs = []
for prog in PyIngestForStreamlit(yaml_string=yaml_string):
    outputs.append(prog)

{} : Reading file 2024-03-02 15:36:46.503699
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battles.csv 0 2024-03-02 15:36:46.505308


{} : Completed file 2024-03-02 15:36:46.594753
{} : Reading file 2024-03-02 15:36:46.595354
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battles.csv 0 2024-03-02 15:36:46.596843
{} : Completed file 2024-03-02 15:36:46.602816
{} : Reading file 2024-03-02 15:36:46.603117
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battles.csv 0 2024-03-02 15:36:46.604184
{} : Completed file 2024-03-02 15:36:46.609417
{} : Reading file 2024-03-02 15:36:46.609671
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battles.csv 0 2024-03-02 15:36:46.610770
{} : Completed file 2024-03-02 15:36:46.616242
{} : Reading file 2024-03-02 15:36:46.616495
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battles.csv 0 2024-03-02 15:36:46.617556
{} : Completed file 2024-03-02 15:36:46.623007
{} : Reading file 2024-03-02 15:36:46.623266
File {} file:.//../../data/csv/got-battles.csv
file:.//../../data/csv/got-battle

In [11]:
outputs

[0.06666666666666667,
 0.13333333333333333,
 0.2,
 0.26666666666666666,
 0.3333333333333333,
 0.4,
 0.4666666666666667,
 0.5333333333333333,
 0.6,
 0.6666666666666666,
 0.7333333333333333,
 0.8,
 0.8666666666666667,
 0.9333333333333333,
 1.0]